In [72]:
import os
from dotenv import load_dotenv
load_dotenv()

True

# The code below is to load the weaviate model and make a reteriver out of it

In [67]:
import os
import weaviate
from weaviate.auth import AuthApiKey
from langchain_weaviate import WeaviateVectorStore  as Weaviate
from langchain_huggingface import HuggingFaceEmbeddings

embeddings= HuggingFaceEmbeddings(model= 'BAAI/bge-large-en-v1.5')


# Load credentials
weaviate_url = os.getenv("WEAVIATE_URL")
weaviate_api_key = os.getenv("WEAVIATE_API_KEY")

# ✅ Correct client initialization for v4
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,                                    # Replace with your Weaviate Cloud URL
    auth_credentials=weaviate_api_key             # Replace with your Weaviate Cloud key
)

# ✅ Initialize LangChain Weaviate vector store
vector_store = Weaviate(
    client=client,
    index_name="InsurancePolicy",
    text_key="text",
    embedding = embeddings
)

reteriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 50}
)

reteriever.invoke('I want to do a knee surgery')


c:\Users\rudra\Desktop\hackathon\hackathon-env\Lib\site-packages\weaviate\warnings.py:292: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
C:\Users\rudra\AppData\Local\Temp\ipykernel_3936\2208518679.py:28: ResourceWarning: unclosed <ssl.SSLSocket fd=5080, family=2, type=1, proto=0, laddr=('192.168.29.3', 59277), raddr=('34.160.174.119', 443)>
  reteriever = vector_store.as_retriever(
c:\Users\rudra\Desktop\hackathon\hackathon-env\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[Document(metadata={'source': 'BAJHLIP23020V012223.pdf', 'page': 46.0, 'type': 'table'}, page_content='| 103 LDR Brachytherapy | 302 Treatment of sesamoid bone fracture |\n| --- | --- |\n| 104 Palliative Radiotherapy | 303 Shoulder arthroscopy / surgery |\n| 105 Radical Radiotherapy | 304 Elbow arthroscopy |\n| 106 Palliative chemotherapy | 305 Amputation of metacarpal bone |\n| 107 Template Brachytherapy | 306 Release of thumb contracture |\n| 108 Neoadjuvant chemotherapy | 307 Incision of foot fascia |\n| 109 Adjuvant chemotherapy | 308 calcaneum spur hydrocort injection |\n| 110 Induction chemotherapy | 309 Ganglion wrist hyalase injection |\n| 111 Consolidation chemotherapy | 310 Partial removal of metatarsal |\n| 112 Maintenance chemotherapy | 311 Repair / graft of foot tendon |\n| 113 HDR Brachytherapy | 312 Revision/Removal of Knee cap |\n| Plastic Surgery | 313 Amputation follow-up surgery |\n| 114 Construction skin pedicle flap | 314 Exploration of ankle joint |\n| 115 Gluteal

# Function and tool for the reteriver function

In [83]:
from typing import List
async def reteriever_tool(query: str):
    reteriever = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 50, 'fetch_k': 500}
    )
    
    related_docs = reteriever.invoke(query)
    
    return related_docs

async def format_docs(docs:List):
    return "\n\n".join([doc.content for doc in docs])

In [100]:
import asyncio
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.agents import AssistantAgent,SocietyOfMindAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
from pydantic import BaseModel,Field

class Response_format(BaseModel):
    decision: str = Field(description = 'Accepted or Rejected')
    amount: int = Field(description = "Amount to be claimed")
    justification: str = Field(description = "The justification regarding the desicion")
    clauses_referenced: List = Field(description = "The clauses related to the acceptance or denial of the claim")

model_client = OpenAIChatCompletionClient(
    model = 'gemini-2.5-flash',
    api_key = os.getenv("GEMINI_API_KEY_2")
)

model_client_4 = OpenAIChatCompletionClient(
    model = 'gemini-2.5-flash',
    api_key = os.getenv("GEMINI_API_KEY"),
    response_format = Response_format
)

model_client_3 = OpenAIChatCompletionClient(
    model = 'gemini-2.5-flash',
    api_key = os.getenv("GEMINI_API_KEY_3")
)

open_router_api_key = os.getenv('OPENROUTER_API')

model_client_1 =  OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-r1:free",
    api_key = open_router_api_key,
    model_info={
        "family":'deepseek',
        "vision" :True,
        "function_calling":True,
        "json_output": False
    }
)


c:\Users\rudra\Desktop\hackathon\hackathon-env\Lib\site-packages\autogen_ext\models\openai\_openai_client.py:453: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [105]:
query_enhancer_system_message = """
    You are the Query Enhancer Agent in a multi-agent document analysis system built for insurance claim evaluation.

    Your role is to take raw, natural language queries — often informal, incomplete, or unstructured — and transform them into clear, semantically rich, and well-structured prompts. These prompts will be used downstream by retrieval and reasoning agents.

    Your enhancement must:
    1. Extract and explicitly mention key entities such as:
    - Age, Gender
    - Medical procedure or illness
    - City or location
    - Policy age or duration
    - Type of policy (if mentioned)
    2. Clarify ambiguous or compressed information where possible.
    3. Maintain the original intent but convert it into a clear, actionable request.
    4. Avoid assumptions — do **not invent details** not present in the original query.
    5. Keep the output **concise, readable, and focused**, optimized for retrieval and validation.

    Respond in a single plain English sentence or two that:
    - Clearly defines what needs to be checked
    - Includes all extracted details that matter for coverage or policy validation
    - Sounds like a professional claim request

    ### Example

    **User Input:**  
    "46-year-old male, knee surgery in Pune, 3-month-old insurance policy"

    **Enhanced Query:**  
    "Check whether a 46-year-old male with a 3-month-old insurance policy is eligible for coverage of knee surgery in Pune."

    Do not include metadata or formatting. Only output the enhanced query.

"""


Retrieval_system_message = """
    You are the Retriever Agent in a multi-agent document analysis system built for insurance claim evaluation.

    🛠 Tools Available:
    1. `retriever_tool` – Use this tool to semantically search and retrieve relevant sections from large unstructured documents such as policy PDFs, contracts, or healthcare manuals.

    🎯 Your Objective:
    You will receive an **Enhanced Query** from the Query Enhancer Agent. Your job is to:
    - Use `retriever_tool` to fetch relevant content from the documents that match the intent and information needs of the enhanced query.
    - Do NOT try to interpret or reason with the content — you are not the decision maker.
    - Your sole task is to **retrieve**, **organize**, and **format** the content in a way that helps the Validator Agent in the next step.

    🧠 How to Format the Retrieved Content:
    1. **Group similar or thematically related clauses together** for clarity.
    For example, if the query involves a surgery, you might group the output as:
    - **Coverage for Surgery**: [Clause 1.2], [Clause 6.5]
    - **Waiting Period**: [Clause 5.1]
    - **Age Restrictions**: [Clause 3.3]
    - **Policy Duration Rules**: [Clause 7.4]

    2. Each group must include the **exact retrieved text** — do NOT paraphrase, summarize, or hallucinate anything.

    3. Label each group clearly using context-aware headings like:
    - **Hospitalization Rules**
    - **Coverage for [Specific Treatment]**
    - **Exclusions**
    - **Pre-existing Disease Clauses**
    - **Eligibility by Age**
    - **Location-based Coverage (if applicable)**
    - **Waiting Period**
    - **Policy Duration Conditions**

    4. If some parts are unrelated, skip them — only show what’s semantically relevant.

    ⚠️ Strict Rules:
    - ❌ Do not invent or hallucinate information.
    - ❌ Do not attempt to validate or make decisions.
    - ✅ Only return what has been retrieved using `retriever_tool`.

    📦 Example Output Format:

    Coverage for Cataract Surgery:

    "Clause 3.4: Cataract surgery is covered after a waiting period of 2 years under all plans."

    Waiting Period:

    "Clause 5.1: A minimum waiting period of 24 months applies to all listed surgeries unless waived."

    Policy Duration:

    "Clause 6.2: Claims are not admissible for elective procedures within the first 90 days of policy inception."



    Your task ends after this. The **Validator Agent** will handle validation and further action. Maintain clarity, structure, and accuracy.
"""

Validator_system_message = """
   You are the Validator Agent in a multi-agent insurance query processing system.  
You now have a dual role — to validate the quality of retrieved content and to make a decision based on it.

Your workflow consists of two phases:

---

🧩 PHASE 1 — VALIDATION:

1. Read the Enhanced Query and extract its key components:
   - Age
   - Medical procedure
   - Location
   - Policy duration or timing
   - Any other relevant attributes

2. Review the retrieved document content (grouped under headings) and check if it:
   - Covers all important elements from the Enhanced Query
   - Is semantically and logically aligned with the query
   - Contains **enough detail** for you to make a decision

3. If the retrieved content is **incomplete** or **irrelevant**, respond with a clear instruction to the **Query Enhancer Agent** on what needs clarification or correction.

---

⚖️ PHASE 2 — DECISION MAKING:

If the retrieved content is valid and sufficient:

4. Read and interpret the relevant clauses **as an insurance analyst or legal advisor would**.

5. Determine whether the claim is **Approved**, **Rejected**, or **Partially Approved** based on:
   - Coverage inclusion
   - Waiting period completion
   - Pre-existing condition clauses
   - Age limits
   - Policy duration rules
   - Exclusions (if any)
   
6. If you Have enough confidence on the evidence you have to accept or reject the claim you can move to step 7.

7.. Your final response must follow this structured JSON format:
```json
{
  "decision": "Approved / Rejected / Partially Approved",
  "amount": [amount or null if not applicable],
  "justification": "A clear explanation using the retrieved clauses",
  "clauses_referenced": [
    "Clause X.Y: [exact quoted clause text]",
    "Clause A.B: [exact quoted clause text]"
  ]
}
Do not hallucinate or assume any information. Only use what is present in the retrieved content.

If something is ambiguous, clearly state that a human review is required — do not fake certainty.

🚫 DO NOT:

Add fake or invented rules

Make assumptions beyond what's retrieved


NOTE : 
After Step 7 is completed RESPOND With : **STOP**
But if at step 6 you feel evidence is not enough Instruct the Enhance Query Agent to write a more enhance query which can help you to determine the result.ArithmeticError




💡 Example Query:
"Is a 58-year-old man with a 2-month-old policy eligible for cataract surgery coverage?"

💡 Expected Output:


{
  "decision": "Rejected",
  "amount": 0,
  "justification": "Cataract surgery is covered, but Clause 5.3 states a 2-year waiting period applies. The policy is only 2 months old.",
  "clauses_referenced": [
    "Clause 5.3: All cataract and other listed surgeries are covered after a minimum waiting period of 24 months."
  ]
}
Respond strictly using the above guidelines and output format.



"""

In [113]:
termination_condition = TextMentionTermination("STOP")

query_enhancer_agent = AssistantAgent(
    name = 'QueryEnhancerAgent',
    model_client = model_client,
    description = "This agent is useful for enhancing the query",
    system_message = query_enhancer_system_message 
)

reterival_agent = AssistantAgent(
    name = 'ReterivalAgent',
    model_client = model_client_3,
    description = "This agent is used to reterieve relevant document",
    system_message = Retrieval_system_message,
    tools = [reteriever_tool]
)

Validator_agent = AssistantAgent(
    name = 'ValidatorAgent',
    model_client = model_client_4,
    description = "This agent is used for validating wether the query is related to the reterieved docs or not.",
    system_message = Validator_system_message
)

team_1 = RoundRobinGroupChat(name="RAGAgentAndValidatorAgent",
                            description = "This agent is used to enhance the user query than reterieve documents and also to validate the documents",
                            participants = [query_enhancer_agent,reterival_agent,Validator_agent],
                            termination_condition = termination_condition,
                            max_turns= 6
                            )

await team_1.reset()
stream = team_1.run_stream(task = 'In the event of delay of the airlines, whilst on a Trip, at any airport specified in the Insured Person’s main trave')
await Console(stream = stream)
# stream = query_enhancer_agent.run_stream(task = "My uncle is 58, he just had a cataract surgery in Nagpur. The policy was bought just 2 months ago. Is it covered?")

# await Console(stream = stream)

---------- TextMessage (user) ----------
In the event of delay of the airlines, whilst on a Trip, at any airport specified in the Insured Person’s main trave


---------- TextMessage (QueryEnhancerAgent) ----------
Check coverage for airline delays for an Insured Person while on a trip at an airport specified in their main travel plan.


c:\Users\rudra\Desktop\hackathon\hackathon-env\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


---------- ToolCallRequestEvent (ReterivalAgent) ----------
[FunctionCall(id='', arguments='{"query":"airline delay coverage for an Insured Person while on a trip at an airport specified in their main travel plan"}', name='reteriever_tool')]
---------- ToolCallExecutionEvent (ReterivalAgent) ----------
[FunctionExecutionResult(content='[Document(metadata={\'source\': \'CHOTGDP23004V012223.pdf\', \'page\': 44.0, \'type\': \'text\'}, page_content=\'CHOLAMANDALAM MS GENERAL INSURANCE COMPANY LIMITED\\nRegistered Office: 2nd Floor, “DARE House”, 2, N.S.C. Bose Road, Chennai – 600 001.\\nToll free: 1800 208 9100, T: +91 (0) 44 4044 5400, F: +91 (0) 44 4044 5550\\nE: customercare@cholams.murugappa.com; website: www.cholainsurance.com\\nIRDA Regn. No.123; PAN AABCC6633K CIN U66030TN2001PLC047977\\nGROUP DOMESTIC TRAVEL INSURANCE\\nCHOTGDP23004V012223\\nPolicy Wordings\\n3. If the Common Carrier is taken out of service on the instruction of the Civil Aviation Authority, or\\nany other governme

c:\Users\rudra\Desktop\hackathon\hackathon-env\Lib\site-packages\autogen_ext\models\openai\_openai_client.py:660: DeprecationWarning: Using response_format to specify the BaseModel for structured output type will be deprecated. Use json_output in create and create_stream instead.
  create_params = self._process_create_args(


---------- TextMessage (ValidatorAgent) ----------
{
  "amount": 0,
  "clauses_referenced": [
    "Endorsement no.18A – FLIGHT DELAY ON BENEFIT BASIS: a. Coverage In the event of delay of the airlines, whilst on a Trip, at any airport specified in the Insured Person’s main travel booking, the policy shall pay a fixed benefit equal to the sum insured as specified in the policy schedule/certificate, for travelling to the next Intended Destination as per Insured person’s main travel booking, if such delay is caused due to any of the following reasons: a. Inclement Weather b. Any Strike, riots, industrial action at the Port or relating to the Common Carrier c. Delay by the Airlines This Benefit shall be payable subject to the following: 1. The Insured Person shall submit to the Company sufficient proof to substantiate the reason for such delay of the Common Carrier, unless this proof is available to the Company directly from a reliable source in the public domain; 2. The delay of the Commo

TaskResult(messages=[TextMessage(id='0f445304-5d49-4d63-b46c-8c5d360bbf02', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 4, 19, 4, 40, 957665, tzinfo=datetime.timezone.utc), content='In the event of delay of the airlines, whilst on a Trip, at any airport specified in the Insured Person’s main trave', type='TextMessage'), TextMessage(id='cd2dc5e7-9e88-42a6-b627-89cc6edf6fad', source='QueryEnhancerAgent', models_usage=RequestUsage(prompt_tokens=368, completion_tokens=24), metadata={}, created_at=datetime.datetime(2025, 8, 4, 19, 4, 45, 460815, tzinfo=datetime.timezone.utc), content='Check coverage for airline delays for an Insured Person while on a trip at an airport specified in their main travel plan.', type='TextMessage'), ToolCallRequestEvent(id='2604b9fa-286f-42a9-b908-fbc1c3146450', source='ReterivalAgent', models_usage=RequestUsage(prompt_tokens=692, completion_tokens=36), metadata={}, created_at=datetime.datetime(2025, 8, 4, 19, 4, 47, 2837